# 20210119
- review image 데이터를 정답지(Y, N)으로 분류하여 별도 디렉토리에 저장

In [ ]:
# 주피터노트북 화면 넓게 보기 위한 용도
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import boto3
import s3fs

import pymysql
from PIL import Image

from dotenv import load_dotenv
import os
load_dotenv() # .env 파일에서 정보를 갖고 오기 위함

In [ ]:
!pip install python-dotenv
# https://medium.com/@namseok.yoo/django-%EC%97%90%EC%84%9C-python-dotenv-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-6d9b58a939ec

In [ ]:
conn = pymysql.connect(database=os.getenv('database'), user=os.getenv('user'), password=os.getenv('password'), host= 'localhost', port=3306)
qry = '''
    SELECT a.goods_no, a.est_no, b.image, a.point_yn
    FROM 
    (
        SELECT goods_no, no as est_no, point_yn
        FROM goods_estimate
        WHERE date_format(regi_date, '%Y%m%d') = '20201201'
        AND type = 'style'
    ) a
    JOIN
    (
        SELECT est_no, image
        FROM goods_estimate_image
        WHERE date_format(rt, '%Y%m%d') = '20201201'
        GROUP BY est_no, image HAVING count(img_no) = 1
    ) b
    ON a.est_no = b.est_no
'''
df = pd.read_sql_query(qry, conn)

In [ ]:
df['point_yn'].describe()

In [ ]:
df.head()

# 20210120
### S3에서 데이터 fetch하기 
- tf.keras.preprocessing.image_dataset_from_directory를 고려했으나 s3에서 바로 동작하지 않아 -> 아래 아티클 참고하여 진행
- https://medium.com/analytics-vidhya/custom-keras-generator-fetching-images-from-s3-to-train-neural-network-4e98694de8ee

In [ ]:
## DB에 저장된 image path를 사용하면 되므로, 다음 과정은 불필요
# import pandas as pd
# import boto3
# from time import time

# bucket_name = 'bucket'
# client = boto3.client('s3')

# test = client.get_paginator('list_objects_v2') # prefix에 특정 key 가져오는거라, 이게 더 복잡한듯
# page_iterator = test.paginate(Bucket=os.getenv('bucket'), Prefix=os.getenv('estimate_prefix')
# g = pd.Series()
# a = time()
# n = 0
# for page in page_iterator:
#     m = [d['Key'] for d in page['Contents'] if d['Key'][-4:] == '.jpg'] 
#     print(m)
#     if len(m) > 0:
#         g = g.append(pd.Series(m))
#     if len(g) == 10000:
#         break

# g.reset_index(drop=True, inplace=True)

In [ ]:
# from keras.preprocessing.image import load_img, img_to_array
# import io

# def fetch_input(path, s3):
#     object = s3.Object(bucket_name, path)
#     img = load_img(io.BytesIO(object.get()['Body'].read()))
#     return(img)


# def preprocess_input(img):
#     image = img.resize((128,128))
#     array = img_to_array(image)
#     return(array)

# # I’m using the keras loading module, but properly the common PIL package can be used as well.

In [ ]:
# def s3_image_generator(files, batch_size = 16):
#     s3 = boto3.resource('s3')
#     while True:
#         batch_paths = np.array(files)
#         batch_input = []
#         batch_output = [0] * len(files)
#         for input_path in batch_paths:
#             input = fectch_input(input_path, s3)
#             input = preprocess_input(input)
#             batch_input += [ input ]
#         batch_x = np.array( batch_input )
#         batch_y = np.array( batch_output )
#         yield( batch_x, batch_y )

In [ ]:
# first = 1
# a = time()
# s3 = boto3.resource('s3')
# for path in g[0:1000]:
#     x_batch=[]
#     object = s3.Object(bucket_name,path)
#     img = load_img(io.BytesIO(object.get()['Body'].read()))
#     image = img.resize((128,128))
#     array = img_to_arrayㅇ(image)
#     preds=(model.predict_on_batch(array))
#     if first==1:
#         predsA=preds.copy()
#         first=0
#     else:
#         predsA=np.append(predsA,preds,axis=0)
# time()-a

s3 -> local -> fetch 밖에 답이 없는듯

- 데이터 디렉토리 아래 클래스별로 이미지 나눠서 저장

In [ ]:
# 아래 코드만 정상동작

def get_image_from_s3(img_path, size=128):
    file = boto3.resource('s3').Object(os.getenv('bucket'), img_path[1:]).get()
    img = Image.open(file['Body']).resize((size, size))
    return img

def save_image(img, path, is_array=False):
    if is_array:
        Image.fromarray(img).save(path)
    else:
        img.save(path)

    return True

def proc_image(img_path, est_no, point_yn):
    img = get_image_from_s3(img_path)
    local_img_path = f'./data/{point_yn.upper()}/{est_no}.jpg'
    _ = save_image(img, local_img_path)
    img_array = np.array(img) # convert image to numpy array
    return img_array

In [ ]:
df.head()

In [ ]:
images_array = df.apply(lambda x: proc_image(x['image'], x['est_no'], x['point_yn']), axis=1) # 해당 path에 없는 이미지가 있나봄 - 마지막에 에러남 